<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/topology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Topology**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

![Normed Vector Space](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/P1S2all.jpg/798px-P1S2all.jpg)

## **Properties of Topological Vector Spaces**

The definition of a topological space relies only upon set theory and is the most general notion of a mathematical space that allows for the definition of concepts such as 

* [continuity](https://en.m.wikipedia.org/wiki/Continuous_function#Continuous_functions_between_topological_spaces): intuitively, take nearby points to nearby points,

* [connectedness](https://en.m.wikipedia.org/wiki/Connected_space): 

* [convergence](https://en.m.wikipedia.org/wiki/Limit_of_a_sequence):

Other spaces, such as manifolds and **metric spaces, are specializations of topological spaces** with extra structures or constraints. 

Being so general, topological spaces are a central unifying notion and appear in virtually every branch of modern mathematics. The branch of mathematics that studies topological spaces in their own right is called [point-set topology](https://en.m.wikipedia.org/wiki/General_topology) or general topology.

https://en.m.wikipedia.org/wiki/Topological_space

https://de.m.wikipedia.org/wiki/Topologischer_Raum

**Point Set (General) Topology**

General topology is the branch of topology that deals with the basic set-theoretic definitions and constructions used in topology. It is the foundation of most other branches of topology, including differential topology, geometric topology, and algebraic topology. Another name for general topology is point-set topology.

The fundamental concepts in point-set topology are continuity, compactness, and connectedness:

* **Continuous functions**, intuitively, take nearby points to nearby points.

* **Compact sets** are those that can be covered by finitely many sets of arbitrarily small size.

* **Connected sets** are sets that cannot be divided into two pieces that are far apart.

https://en.m.wikipedia.org/wiki/General_topology

## **Invariants of Topological Spaces**

https://en.m.wikipedia.org/wiki/Fundamental_group

### **Topologische Invarianten (Properties)**

1. Anzahl der Zusammenhangs-Komponenten

2. Isomorphie-Klasse der Fundamentalgruppe

3. Euler-Charakteristik

4. Geschlecht einer Fläche

5. Bettizahlen eines topologischen Raumes

https://de.m.wikipedia.org/wiki/Topologische_Invariante

https://en.m.wikipedia.org/wiki/Topological_property

https://de.m.wikipedia.org/wiki/Geometrisierung_von_3-Mannigfaltigkeiten

https://de.m.wikipedia.org/wiki/Differentialtopologie

### **Homotopy Groups**

* Homotopy groups are used in algebraic topology to classify topological spaces. 

* The first and simplest homotopy group is the fundamental group, which records information about loops in a space. Intuitively, homotopy groups record information about the basic shape, or holes, of a topological space.

https://en.m.wikipedia.org/wiki/Homotopy_group

https://de.m.wikipedia.org/wiki/Homotopie

### **Fundamental Group**

* The fundamental group of a topological space is the group of the equivalence classes under homotopy of the loops contained in the space. It records information about the basic shape, or holes, of the topological space. 

* The fundamental group is the first and simplest homotopy group. The fundamental group is a homotopy invariant—topological spaces that are homotopy equivalent (or the stronger case of homeomorphic) have isomorphic fundamental groups.

### **Homology Groups**

Die Homologie ist ein mathematischer Ansatz, die Existenz von Löchern zu formalisieren. Gewisse „sehr feine“ Löcher sind für die Homologie unsichtbar; hier kann u. U. auf die schwerer zu bestimmenden Homotopiegruppen zurückgegriffen werden.



https://de.m.wikipedia.org/wiki/Homologietheorie

### **Triangulierung**

https://de.m.wikipedia.org/wiki/Triangulierung_(Topologie)

## **Classification of Topological Spaces**

https://en.m.wikipedia.org/wiki/History_of_the_separation_axioms

#### **Kolmogorov Space (T<sub>0</sub>)**

A space is Kolmogorov if for every pair of distinct points x and y in the space, there is at least either an open set containing x but not y, or an open set containing y but not x.

If for every pair of distinct points of X, at least one of them has a neighborhood not containing the other. In a T0 space, all points are topologically distinguishable.

https://en.m.wikipedia.org/wiki/Kolmogorov_space

#### **Fréchet Space (T<sub>1</sub>)**

Fréchet spaces are generalizations of Banach spaces (normed vector spaces that are complete with respect to the metric induced by the norm).

https://en.m.wikipedia.org/wiki/Fréchet_space

Instead, with the topology of compact convergence, C(a, b) can be given the structure of a Fréchet space: a locally convex topological vector space whose topology can be induced by a complete translation-invariant metric.

https://en.m.wikipedia.org/wiki/Locally_convex_topological_vector_space

#### **Haussdorf Spaces (T<sub>2</sub>)**

* A topological space is Hausdorff if and only if it is both preregular (i.e. topologically distinguishable points are separated by neighbourhoods) and Kolmogorov (i.e. distinct points are topologically distinguishable). A topological space is preregular if and only if its Kolmogorov quotient is Hausdorff.

* Almost all spaces encountered in analysis are Hausdorff; most importantly, the real numbers (under the standard metric topology on real numbers) are a Hausdorff space. More generally, all metric spaces are Hausdorff. In fact, many spaces of use in analysis, such as topological groups and topological manifolds, have the Hausdorff condition explicitly stated in their definitions.

https://en.m.wikipedia.org/wiki/Topological_vector_space

https://en.m.wikipedia.org/wiki/Hausdorff_space

## **Umgebung (Neighbourhood)**

* a neighbourhood (or neighborhood) is one of the basic concepts in a topological space. It is closely related to the concepts of open set and interior. 

* Intuitively speaking, a neighbourhood of a point is a set of points containing that point where one can move some amount in any direction away from that point without leaving the set.

https://de.m.wikipedia.org/wiki/Umgebung_(Mathematik)

https://en.m.wikipedia.org/wiki/Functional_analysis

https://en.m.wikipedia.org/wiki/Geometric_measure_theory